# Delta Lake Lab 
## Unit 5: Schema Enforcement & Evolution

In the previous unit, we -
1. Did a delete operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log
2. Did an insert operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log
3. Did an update operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log
4. Did a merge (upsert) operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log

In this unit, we will -
1. Study schema enforcement
2. And schema evolution possible with delta lake


### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/11/01 22:00:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [ ]:
ACCOUNT_NAME = "akhjain"

In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"

In [7]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-10887990-f51d-46ad-a768-2f8135d5fd95-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-2517e153-6923-4599-8b2e-5746fcf10973-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-363ee39f-ff75-4e4d-8938-d866d43955ca-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-99635b87-214b-4afa-895e-8c124bf54009-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-9a73bff5-5ed8-4ce2-9383-b6f2865cbc6a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-bd1648aa-3db7-43e9-beb7-db0593eefbb5-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-buc

In [8]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

6


In [9]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

6


### 4. Existing schema

In [10]:
spark.sql("DESCRIBE FORMATTED loan_db.loans_by_state_delta").show(truncate=False)

ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+----------------------------+---------------------------------------------------+-------+
|col_name                    |data_type                                          |comment|
+----------------------------+---------------------------------------------------+-------+
|addr_state                  |string                                             |       |
|count                       |bigint                                             |       |
|                            |                                                   |       |
|# Partitioning              |                                                   |       |
|Not partitioned             |                                                   |       |
|                            |                                                   |       |
|# Detailed Table Information|                                                   |       |
|Name                        |loan_db.loans_by_state_delta                       |       |

### 5. Attempt to modify the schema

In [11]:
# Add a column called collateral_value
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from loan_db.loans_by_state_delta")
schemaEvolvedDF.show(3)

+----------+-----+-----------------+
|addr_state|count| collateral_value|
+----------+-----+-----------------+
|        AK|    0|1709.497137955568|
|        AL|    0|8051.143958005459|
|        AR|    0|5775.925576589018|
+----------+-----+-----------------+
only showing top 3 rows



In [12]:
# Attempt to append to the table
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 8ac45cea-8820-4a01-b29d-3a2311ddc86b).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)


Data schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)
-- collateral_value: double (nullable = true)

         

### 6. Supply "mergeSchema" option

In [13]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

6


In [14]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

6


In [15]:
schemaEvolvedDF.write.option("mergeSchema",True).format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)

In [16]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta").show(truncate=False)

+----------+-----+--------------------+
|addr_state|count|collateral_value    |
+----------+-----+--------------------+
|AK        |0    |1709.497137955568   |
|AL        |0    |8051.143958005459   |
|AR        |0    |5775.925576589018   |
|AZ        |0    |9476.047869880924   |
|CA        |2584 |2.5846787948188066E7|
|CO        |0    |3666.4222617947817  |
|CT        |0    |8078.688178371882   |
|DC        |0    |7135.143433452461   |
|DE        |0    |7195.325566306054   |
|FL        |0    |3133.5292311175454  |
|GA        |0    |8062.503712025726   |
|HI        |0    |1081.4914646176655  |
|IA        |186  |1866039.3042891505  |
|ID        |0    |8133.304803837666   |
|IL        |0    |4764.94287381709    |
|IN        |3497 |3.4978374898949035E7|
|KS        |0    |9701.253460019921   |
|KY        |0    |6232.167713919952   |
|LA        |0    |5089.687568245219   |
|MA        |0    |5467.504094508642   |
+----------+-----+--------------------+
only showing top 20 rows



In [17]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

7


In [18]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

7


In [19]:
# Lets look at our datalake and changes from the above execution
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-10887990-f51d-46ad-a768-2f8135d5fd95-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-2517e153-6923-4599-8b2e-5746fcf10973-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-363ee39f-ff75-4e4d-8938-d866d43955ca-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-99635b87-214b-4afa-895e-8c124bf54009-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-9a73bff5-5ed8-4ce2-9383-b6f2865cbc6a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-b34ca7fb-09ae-455b-bd45-c6e5681060d2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-bd1648aa-3db7-43e9-beb7-db0593eefbb5-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumabl

There is one extra parquet, containing the data with the new column, and one new transaction log


Lets look at the log-

In [20]:
!gsutil cat {DATA_LAKE_ROOT_PATH}/delta-consumable/_delta_log/00000000000000000006.json

{"metaData":{"id":"8ac45cea-8820-4a01-b29d-3a2311ddc86b","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"addr_state\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"count\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"collateral_value\",\"type\":\"double\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1667338219033}}
{"add":{"path":"part-00000-b34ca7fb-09ae-455b-bd45-c6e5681060d2-c000.snappy.parquet","partitionValues":{},"size":1687,"modificationTime":1667340173884,"dataChange":true,"stats":"{\"numRecords\":51,\"minValues\":{\"addr_state\":\"AK\",\"count\":0,\"collateral_value\":140.9711060318264},\"maxValues\":{\"addr_state\":\"WY\",\"count\":3497,\"collateral_value\":3.4978374898949035E7},\"nullCount\":{\"addr_state\":0,\"count\":0,\"collateral_value\":0}}"}}
{"commitInfo":{"timestamp":1667340174129,"operation":"WRITE","operationParameters":

In [21]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log


gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json


In [22]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

9


In [23]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

8


In [24]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)


gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000008.json


In [28]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000008.json


In [29]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part*

gs://dll-data-bucket-885979867746/delta-consumable/part-00000-10887990-f51d-46ad-a768-2f8135d5fd95-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-2517e153-6923-4599-8b2e-5746fcf10973-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-363ee39f-ff75-4e4d-8938-d866d43955ca-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-99635b87-214b-4afa-895e-8c124bf54009-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-9a73bff5-5ed8-4ce2-9383-b6f2865cbc6a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-b28e7cf4-80c3-4054-8cef-3cff4fe805ea-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-b34ca7fb-09ae-455b-bd45-c6e5681060d2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-bd1648aa-3db7-43e9-beb7-db0593eefbb5-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumab

In [27]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

13


In [26]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

9


In [30]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000008.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/0000000000000000000

In [31]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

17


In [32]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

10


In [33]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000008.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/0000000000000000000

In [34]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

23


In [35]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

11


In [36]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000008.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/0000000000000000000

In [37]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

29


In [38]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

12


Note how there is a parquet file created - this is a compacted version of table changes 0-9
Delta will load this parquet into memory, to avoid having to read too many small files
At any given point of time, delta will not read more than 10 json transaction logs, but will read all the parquet transaction logs.

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK